<a href="https://colab.research.google.com/github/olya-ibragimova/hello-github/blob/master/All_components_pipeline_Data8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
# Uses python3
# import matplotlib
# matplotlib.use('Agg')
import numpy as np
import pandas as pd
import h5py
from datetime import datetime
import os
import scipy
from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau, LearningRateScheduler, Callback, ModelCheckpoint
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
from keras.utils import multi_gpu_model
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter

from google.colab import drive

plt.rcParams.update({'font.size': 22})

drive.mount('/content/drive/')

range_curves = 250
num_of_points = 250
out = "s22"
patience = 400
a = 6

# Loading data
print("Loading data")
data = np.asarray(h5py.File('/content/drive/My Drive/Datasets/Data8/data8_train.h5', 'r')['data'])
print(data.shape)
data_test = np.asarray(h5py.File('/content/drive/My Drive/Datasets/Data8/data8_test.h5', 'r')['data'])
print(data_test.shape)

# Loading normalization parameters
data_max = np.loadtxt('/content/drive/My Drive/Datasets/Data8/data8_max.csv', delimiter=',')
data_min = np.loadtxt('/content/drive/My Drive/Datasets/Data8/data8_min.csv', delimiter=',')

# Some helper variables:
max_y_lim = [350,
            1, 1, 1, 1, 1, 1, 1, 1, 1,
            950, 900, 900, 750, 550, 550,
            0.25]
min_y_lim = [0,
            -1, -1, -1, -1, -1, -1, -1, -1, -1,
            -950, -900, -900, -750, -550, -550,
            0]
units = [" MPa",
        "", "", "", "", "", "", "", "", "",
        " MPa", " MPa", " MPa", " MPa", " MPa", " MPa",
        ""]

from_ = []
to_ = []

out_ = ['g',
    'r1',  'r2',  'r3',  'r4',  'r5',  'r6',  'r7',  'r8',  'r9',
    's11', 's22', 's33', 's12', 's13', 's23',
    'ashear']

c = 0
for i in out_:
    from_.append(15+c*num_of_points)
    to_.append(15+(c+1)*num_of_points)
    c+=1
    
print(from_)
print(to_)

activation = 'tanh'
metrics = ['mse', 'mape']
optimizer = 'adam'
loss = 'mean_squared_error'
kernel_init =  'glorot_uniform'

# for out in  ['g',
#       'r1',  'r2',  'r3',  'r4',  'r5',  'r6',  'r7',  'r8',  'r9',
#       's11', 's22', 's33', 's12', 's13', 's23',
#       'ashear']:  

index_ = out_.index(out)

cols = ['tau0','h0',
    'd11', 'd22', 'd33', 'd12',
    'r1_init','r2_init', 'r3_init', 'r4_init', 
    'r5_init', 'r6_init', 'r7_init', 'r8_init', 'r9_init']

print("input:	",cols[:cols.index('r9_init') + 1])
print("output:	 {} curve ({} values)".format(out, -from_[index_]+to_[index_]))
print(out, index_, from_[index_], to_[index_])

# X & Y test
X_test = data_test[:,:cols.index('r9_init') + 1]
y_test = data_test[:,from_[index_]:to_[index_]][:,::a]


# samples_in_training_data = 120000
permutation = list(np.random.permutation(data.shape[0]))
data_temp = data[permutation,:]
# data_temp = data_temp[:int(samples_in_training_data),:]

# X & Y train
X_train = data_temp[:,:cols.index('r9_init') + 1]
y_train = data_temp[:,from_[index_]:to_[index_]][:,::a]
# Setting up some hyperparameters:


# Model:

model = Sequential()
model.add(Dense(1024, input_dim=X_train.shape[1], kernel_initializer=kernel_init, activation=activation))
model.add(Dropout(0.3))
model.add(Dense(1024, kernel_initializer = kernel_init, activation = activation))
model.add(Dense(1024, kernel_initializer = kernel_init, activation = activation))
model.add(Dropout(0.3))
model.add(Dense(512, kernel_initializer = kernel_init, activation = activation))
model.add(Dense(512, kernel_initializer = kernel_init, activation = activation))
model.add(Dense(512, kernel_initializer = kernel_init, activation = activation))
model.add(Dropout(0.3))
model.add(Dense(256, kernel_initializer = kernel_init, activation = activation))
model.add(Dense(256, kernel_initializer = kernel_init, activation = activation))
model.add(Dense(256, kernel_initializer = kernel_init, activation = activation))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], kernel_initializer='normal'))
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

name_ckpt = 'checkpoint_data8_0706_{}.h5'.format(out)

checkpoint = ModelCheckpoint(name_ckpt, 
                            monitor='val_mse', 
                            verbose=1, save_best_only=True, mode='min')
batch_size = 2**13

callbacks_list = [checkpoint]

hist = model.fit(
    X_train, y_train, 
    epochs=patience, 
    batch_size=batch_size, 
    callbacks=callbacks_list, 
    validation_split=0.15, 
    shuffle=True, verbose = 2)

model = load_model(name_ckpt)
print('Best val_score model is loaded'.format(out))
preds = model.predict(X_test, batch_size=8196)

print(X_test.shape)
print("test mse:	",mean_squared_error(y_test, preds))

data_max_x = data_max[:cols.index('r9_init') + 1]
data_min_x = data_min[:cols.index('r9_init') + 1]
    
data_max_y = data_max[from_[index_]:to_[index_]][::a]
data_min_y = data_min[from_[index_]:to_[index_]][::a]

y_test = (y_test + 1)*(data_max_y - data_min_y)/2 + data_min_y
preds =  (preds  + 1)*(data_max_y - data_min_y)/2 + data_min_y
X_test = (X_test + 1)*(data_max_x - data_min_x)/2 + data_min_x

print("test MSE:	",mean_squared_error(y_test, preds))
print("test RMSE:	",np.sqrt(mean_squared_error(y_test, preds)))

figs_ = '/content/drive/My Drive/Figures/Data8/figs_data8_{}_4'.format(out)

if not os.path.isdir(figs_):
    os.mkdir(figs_)
    print(figs_ + " has come into existence".format(out))

plt.figure()
fig, _ = plt.subplots(figsize=(15, 9))
plt.semilogy(hist.history['mse'],linewidth=4, label = 'On training set')
plt.semilogy(hist.history['val_mse'],linewidth=4, label = 'On validation set', alpha = 0.8)
plt.semilogy(savgol_filter(hist.history['val_mse'], 51, 3),linewidth=4, label = 'On validation set', alpha = 0.8)
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.title('Model')
plt.xlim(0,patience+5)
plt.grid(which = 'both', linestyle = ':')
plt.legend()
plt.savefig(figs_+'/mse_{0}.png'.format(out))
plt.show()


plt.figure()
fig, _ = plt.subplots(figsize=(15, 9))
plt.semilogy(hist.history['mape'], label = 'On training set')
plt.semilogy(hist.history['val_mape'], label = 'On validation set', alpha = 0.8)
plt.semilogy(savgol_filter(hist.history['val_mape'], 51, 3), label = 'On validation set (smooth)', alpha = 0.8)
plt.xlabel('Epochs')
plt.ylabel('MAPE')
plt.title('Model')
plt.xlim(0,patience+5)
plt.grid(which = 'both', linestyle = ':')
plt.legend()
plt.savefig(figs_+'/mape_{0}.png'.format(out))
plt.show()

print("Plotting curves... ")
for curves in range(range_curves):
    # print(X_test[curves, :])
    plt.figure(figsize=(20,10))
    plt.plot(y_test[curves,:], "o-", linewidth=4, label = "CP Simulation")
    plt.plot(preds[curves,:], "s-", linewidth=4, alpha = 0.8, label = "ML Prediction")
    plt.plot([], linewidth=0, label="RMSE = {0:1.2f} {1}".format(
        np.sqrt(mean_squared_error(y_test[curves,:],preds[curves,:])),
                units[out_.index(out)]))
    plt.grid(color='gray', linestyle=':', linewidth=0.5)
    plt.title("Test set, ML prediction: {}".format(out))
    plt.ylim((min_y_lim[out_.index(out)], max_y_lim[out_.index(out)]))
    plt.legend()
    plt.savefig(figs_+'/res_{0}_{1}.png'.format(out,curves))
    # print('/res_{0}_{1}.png'.format(out,curves))
    plt.close()

print("Complete.")

Using TensorFlow backend.


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Loading data
(154888, 4265)
(17209, 4265)
[15, 265, 515, 765, 1015, 1265, 1515, 1765, 2015, 2265, 2515, 2765, 3015, 3265, 3515, 3765, 4015]
[265, 515, 765, 1015, 1265, 1515, 1765, 2015, 2265, 2515, 2765, 3015, 3265, 3515, 3765, 4015, 4265]
input:	 ['tau0', 'h0', 'd11', 'd22', 'd33', 'd12', 'r1_init', 'r2_init', 'r3_init', 'r4_init', 'r5_init', 'r6_init', 'r7_init', 'r8_init', 'r9_init']
output:	 s22 curve (250 values)
s22 11 2765 3015
Train on 131654 samples, validate on 23234 samples
Epoch 1/400
 - 4s - loss: 0.0901 - mse: 0.0901 - mape: 4238.8442 - val_loss: 0.0337 - val_mse: 0.0337 - val_mape: 3429.9592

Epoch 00001: val_mse improved from inf to 0.03366, saving model to checkpoint_data8_0706_s22.h5
Epoch 2/400
 - 2s - loss: 0.0458 - mse: 0.0458 - mape: 3471.0815 - val_loss: 0.0316 - val_mse: 0.0316 - val_mape: 2904.2473

Epoch 00002: val_mse improved fro